# Demonstrate Statcasting

A notebook of a few ways to use the Statcast pitching tools.

In [1]:
# boilerplate imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

from random import randrange
import scipy.stats as sst

# ipython reload capabilities for edits to external code
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

import pandas as pd



In [2]:
# start by selecting your year of interest
year = '2020'

# todo: add cross-year slices

In [3]:
# pull in the codes for statcast queries

PlayerDict = dict()
f = open('data/playerdict'+year+'.txt')

line = f.readline()
while line:
    PlayerDict[line.split(',')[0].strip()] = line.split(',')[1].strip()
    line = f.readline()

f.close()
print(PlayerDict['Jose Berrios'])

621244


In [4]:

check_players = ['Jose Berrios','Zack Greinke']

save = True

verbose=False
BPDict = {}

for pindx,pnumber in enumerate(check_players):
    
    if verbose: print(pindx,' ',pnumber)
    
    link = 'https://baseballsavant.mlb.com/statcast_search/csv?all=true&hfPT=&hfAB=&hfBBT=&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfGT=R%7C&hfC=&hfSea='+year+'%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfInfield=&team=&position=&hfOutfield=&hfRO=&home_road=&hfFlag=&hfPull=&pitchers_lookup%5B%5D='+str(PlayerDict[pnumber])+'&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name&sort_col=pitches&player_event_sort=h_launch_speed&sort_order=desc&min_pas=0&type=details&'

    BPDict[pnumber] = pd.read_csv(link, low_memory=False)
    
    if save:
        BPDict[pnumber].to_csv('data/'+pnumber.replace(" ","")+'_'+year+'.csv',index=False,encoding='utf-8')





In [5]:

# here's everything we have access to now!
BPDict[plr].keys()

NameError: name 'plr' is not defined

In [ ]:
plr = check_players[0] # select Berrios



In [ ]:
starts = np.unique(BPDict[plr]['game_date'])
pitches = np.unique(BPDict[plr]['pitch_type'])

markers = ['s','v','X','p','D','o']
print(starts)

for istart,start in enumerate(starts):
    which_pitches = np.where(BPDict[plr]['game_date']==start)[0]
    for ipitch,pitch in enumerate(pitches):
        speeds = np.array(BPDict[plr]['release_speed'][which_pitches])
        ptypes = BPDict[plr]['pitch_type'][which_pitches]
        agreed = np.where(ptypes == pitch)[0]
        if ipitch==0:
            plt.plot(np.arange(0,agreed.size,1),speeds[agreed],lw=1.0,color=cm.magma(istart/len(starts),1.),label=start)
        else:
            plt.plot(np.arange(0,agreed.size,1),speeds[agreed],lw=1.0,color=cm.magma(istart/len(starts),1.))


        if istart==0:
            plt.scatter(np.arange(0,agreed.size,1),speeds[agreed],marker=markers[ipitch],s=20.,color=cm.magma(istart/len(starts),1.),label=pitch)
        else:
            plt.scatter(np.arange(0,agreed.size,1),speeds[agreed],marker=markers[ipitch],s=20.,color=cm.magma(istart/len(starts),1.))



plt.legend()
plt.xlabel('Pitch Number',size=12)
plt.ylabel('Velocity')
plt.tight_layout()

plt.savefig('figures/'+plr.replace(" ","")+'.png',dpi=300)

In [ ]:

# check the outcomes


plt.scatter(BPDict[plr]['launch_speed'],BPDict[plr]['launch_angle'],color='black',s=4.)

plt.plot([40.,120.],[0,0],color='grey',linestyle='dashed')

plt.axis([40.,120.,-55,90])
plt.ylabel('Lauch Angle',size=12)
plt.xlabel('Launch Speed',size=12)

In [ ]:

# check a particular pitch

print(np.unique(BPDict[plr]['pitch_type']))
pitchtype = 'FF'

pitch_velocity = BPDict[plr]['release_speed'][BPDict[plr]['pitch_type']==pitchtype]
pitch_spin = BPDict[plr]['release_spin_rate'][BPDict[plr]['pitch_type']==pitchtype]
pitch_inning = np.array(BPDict[plr]['inning'][BPDict[plr]['pitch_type']==pitchtype]).astype('float')


plt.scatter(pitch_velocity,pitch_spin,color=cm.viridis((pitch_inning-1)/8.,1.),s=10.)

plt.xlabel('Pitch Velocity',size=18)
plt.ylabel('Pitch Spin Rate',size=18)
plt.tight_layout()

